# Accion Microsft

In [6]:
import requests
import pandas as pd
import plotly.express as px

API_KEY = "f8t2eKKxjdTt7LiuD0Na5nMB0aYdpdgi"
TICKER = "MSFT"

# Nuevo endpoint “stable” para precios EOD completos (open, high, low, close, volumen, etc.)
url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={TICKER}&apikey={API_KEY}"

headers = {
    "User-Agent": "Mozilla/5.0"
}

resp = requests.get(url, headers=headers)
if resp.status_code != 200:
    print("ERROR:", resp.status_code)
    print("Respuesta del servidor:", resp.text)
    raise RuntimeError(f"Error al conectar con FMP (stable endpoint): {resp.status_code}")

data = resp.json()

# aquí puede que la estructura JSON sea distinta: por ejemplo “historical” dentro de un campo “historical” o directamente raíz
if "historical" in data:
    hist = data["historical"]
else:
    # si la respuesta es directamente un arreglo
    hist = data

df = pd.DataFrame(hist)
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date")

fig = px.line(df, x="date", y="close",
              title=f"Precio de cierre diario — {TICKER} (endpoint stable)",
              labels={"date": "Fecha", "close": "Precio (USD)"},
              template="plotly_white")
fig.show()

display(df.head(), df.tail())


,symbol,date,open,high,low,close,volume,change,changePercent,vwap
1254,MSFT,2020-10-19,220.42,222.30,213.72,214.22,27625841,-6.19,-2.81000,217.6650
1253,MSFT,2020-10-20,215.80,217.37,213.09,214.65,22753511,-1.15,-0.53290,215.2275
1252,MSFT,2020-10-21,213.12,216.92,213.12,214.80,22724906,1.68,0.78829,214.4900
1251,MSFT,2020-10-22,213.93,216.06,211.70,214.89,22351500,0.96,0.44874,214.1450
1250,MSFT,2020-10-23,215.03,216.28,213.16,216.23,18879608,1.20,0.55806,215.1750


,symbol,date,open,high,low,close,volume,change,changePercent,vwap
4,MSFT,2025-10-10,519.64,523.58,509.63,510.96,24133840,-8.68,-1.67000,515.9525
3,MSFT,2025-10-13,516.41,516.41,511.68,514.05,14284238,-2.36,-0.45700,514.6375
2,MSFT,2025-10-14,510.23,515.28,506.00,513.57,14684300,3.35,0.65461,511.2700
1,MSFT,2025-10-15,514.96,517.19,510.00,513.43,14694700,-1.53,-0.29711,513.8950
0,MSFT,2025-10-16,512.58,516.85,508.13,511.61,15060336,-0.97,-0.18924,512.2925


In [7]:
df = df[['close','date']]
df['price'] = df['close']
df.drop(columns=['close'], inplace=True)
df.set_index('date', inplace=True)
df

,price
date,
2020-10-19,214.22
2020-10-20,214.65
2020-10-21,214.80
2020-10-22,214.89
2020-10-23,216.23
...,...
2025-10-10,510.96
2025-10-13,514.05
2025-10-14,513.57


# Scaler

In [8]:
from sklearn.preprocessing import MinMaxScaler

In [9]:
scaler = MinMaxScaler()
price_scaled = scaler.fit_transform(df[['price']])
df_scaled = pd.DataFrame({'Date': df.index, 'price_scaled': price_scaled.flatten()})
df_scaled.set_index('Date', inplace=True)
df_scaled

,price_scaled
Date,
2020-10-19,0.035672
2020-10-20,0.036963
2020-10-21,0.037413
2020-10-22,0.037683
2020-10-23,0.041703
...,...
2025-10-10,0.925955
2025-10-13,0.935225
2025-10-14,0.933785


# Window

In [10]:
import numpy as np
import pandas as pd
from typing import Tuple, Optional

def make_sliding_windows_pd(
    data: pd.Series | pd.DataFrame,
    col: Optional[str] = None,
    lags: int = 30,
    horizon: int = 1,
    step: int = 1,
    return_index: bool = True
) -> tuple:
    """
    Ventanas deslizantes a partir de una Serie/DataFrame con índice temporal.

    Parámetros
    ----------
    data : pd.Series | pd.DataFrame
        Serie univariada o DataFrame con una columna objetivo.
    col : str | None
        Nombre de la columna si 'data' es DataFrame. Ignorado si es Series.
    lags : int
        Tamaño de ventana (número de rezagos).
    horizon : int
        Pasos futuros a predecir (1 = next-step; >1 = multi-step directo).
    step : int
        Avance entre ventanas consecutivas.
    return_index : bool
        Si True, devuelve índices de fin de X y de y (útil para alinear fechas).

    Returns
    -------
    X : np.ndarray, shape (n_samples, lags)
    y : np.ndarray, shape (n_samples, horizon)
    x_end_index : pd.Index (opcional)
        Índice (fechas) del último punto incluido en cada ventana X.
    y_end_index : pd.Index (opcional)
        Índice (fechas) del último target de cada y (t+lags+horizon-1).
    """
    # Selección de serie
    if isinstance(data, pd.DataFrame):
        if col is None:
            if data.shape[1] != 1:
                raise ValueError("Pasa 'col' si el DataFrame tiene >1 columna.")
            series = data.iloc[:, 0]
        else:
            series = data[col]
    else:
        series = data

    # Orden y limpieza mínima
    series = series.sort_index()
    series = series.dropna()

    idx = series.index
    s = series.to_numpy(dtype=float)
    n = len(s)

    max_start = n - lags - horizon
    if max_start < 0:
        empty_X = np.empty((0, lags), dtype=float)
        empty_y = np.empty((0, horizon), dtype=float)
        return (empty_X, empty_y, idx[:0], idx[:0]) if return_index else (empty_X, empty_y)

    starts = np.arange(0, max_start + 1, step, dtype=int)
    X = np.empty((len(starts), lags), dtype=float)
    y = np.empty((len(starts), horizon), dtype=float)

    # Índices de referencia
    x_end_index = idx[starts + lags - 1]
    y_end_index = idx[starts + lags + horizon - 1]

    for i, st in enumerate(starts):
        X[i] = s[st : st + lags]
        y[i] = s[st + lags : st + lags + horizon]

    return (X, y, x_end_index, y_end_index) if return_index else (X, y)


In [11]:
X, y, x_idx, y_idx = make_sliding_windows_pd(
    data=df_scaled, col="price_scaled",
    lags=5, horizon=1, step=1, return_index=True
)

# Train/Test

In [12]:
n = len(X)
n_train = int(n * 0.80)
n_val   = int(n * 0.10)
n_test  = n - n_train - n_val

X_train, y_train = X[:n_train], y[:n_train]
X_val,   y_val   = X[n_train:n_train+n_val], y[n_train:n_train+n_val]
X_test,  y_test  = X[-n_test:], y[-n_test:]

In [13]:
display(X_train, y_train)
print(f'Train shape {X_train.shape}')
display(X_val, y_val)
print(f'Val shape {X_val.shape}')
display(X_test, y_test)
print(f'Test shape {X_test.shape}')

array([[0.0356725 , 0.03696259, 0.03741262, 0.03768264, 0.04170292],
       [0.03696259, 0.03741262, 0.03768264, 0.04170292, 0.02325163],
       [0.03741262, 0.03768264, 0.04170292, 0.02325163, 0.03276229],
       ...,
       [0.62167352, 0.6371846 , 0.64543518, 0.64057484, 0.64201494],
       [0.6371846 , 0.64543518, 0.64057484, 0.64201494, 0.65047553],
       [0.64543518, 0.64057484, 0.64201494, 0.65047553, 0.64927545]])

array([[2.32516276e-02],
       [3.27622934e-02],
       [1.05007351e-03],
       [7.17050194e-03],
       [4.20029402e-04],
       [0.00000000e+00],
       [1.23008611e-02],
       [4.21829528e-02],
       [6.28844019e-02],
       [6.41744922e-02],
       [4.81833728e-02],
       [2.60418229e-02],
       [4.26629864e-02],
       [3.93327533e-02],
       [4.25429780e-02],
       [4.47031292e-02],
       [3.63925475e-02],
       [2.62518376e-02],
       [3.02721190e-02],
       [2.41816927e-02],
       [2.33416339e-02],
       [3.45924215e-02],
       [3.46224236e-02],
       [3.87027092e-02],
       [3.52224656e-02],
       [4.16429150e-02],
       [3.91227386e-02],
       [3.57325013e-02],
       [3.60925265e-02],
       [3.58825118e-02],
       [4.10428730e-02],
       [2.84119888e-02],
       [2.45717200e-02],
       [3.27922955e-02],
       [3.56124929e-02],
       [3.54024782e-02],
       [5.08535597e-02],
       [5.12735892e-02],
       [4.87834148e-02],
       [6.07842549e-02],


Train shape (1000, 5)


array([[0.64057484, 0.64201494, 0.65047553, 0.64927545, 0.6414149 ],
       [0.64201494, 0.65047553, 0.64927545, 0.6414149 , 0.64321503],
       [0.65047553, 0.64927545, 0.6414149 , 0.64321503, 0.64753533],
       [0.64927545, 0.6414149 , 0.64321503, 0.64753533, 0.64939546],
       [0.6414149 , 0.64321503, 0.64753533, 0.64939546, 0.67558729],
       [0.64321503, 0.64753533, 0.64939546, 0.67558729, 0.66685668],
       [0.64753533, 0.64939546, 0.67558729, 0.66685668, 0.66724671],
       [0.64939546, 0.67558729, 0.66685668, 0.66724671, 0.67750743],
       [0.67558729, 0.66685668, 0.66724671, 0.67750743, 0.6728271 ],
       [0.66685668, 0.66724671, 0.67750743, 0.6728271 , 0.68890822],
       [0.66724671, 0.67750743, 0.6728271 , 0.68890822, 0.69064835],
       [0.67750743, 0.6728271 , 0.68890822, 0.69064835, 0.61210285],
       [0.6728271 , 0.68890822, 0.69064835, 0.61210285, 0.62416369],
       [0.68890822, 0.69064835, 0.61210285, 0.62416369, 0.61843329],
       [0.69064835, 0.61210285, 0.

array([[0.64321503],
       [0.64753533],
       [0.64939546],
       [0.67558729],
       [0.66685668],
       [0.66724671],
       [0.67750743],
       [0.6728271 ],
       [0.68890822],
       [0.69064835],
       [0.61210285],
       [0.62416369],
       [0.61843329],
       [0.62743392],
       [0.65359575],
       [0.66934685],
       [0.66067625],
       [0.6470853 ],
       [0.66214635],
       [0.66865681],
       [0.67372716],
       [0.63805466],
       [0.64033482],
       [0.64642525],
       [0.63952477],
       [0.63166422],
       [0.64405508],
       [0.64942546],
       [0.67702739],
       [0.66202634],
       [0.66343644],
       [0.68599802],
       [0.68665807],
       [0.70531937],
       [0.72092046],
       [0.72377066],
       [0.73112118],
       [0.72305061],
       [0.7400318 ],
       [0.74174192],
       [0.73487144],
       [0.74783235],
       [0.75644295],
       [0.70522937],
       [0.70414929],
       [0.7028592 ],
       [0.69880892],
       [0.711

Val shape (125, 5)


array([[0.55839909, 0.55647895, 0.55023852, 0.50787555, 0.49638475],
       [0.55647895, 0.55023852, 0.50787555, 0.49638475, 0.47040293],
       [0.55023852, 0.50787555, 0.49638475, 0.47040293, 0.49350455],
       [0.50787555, 0.49638475, 0.47040293, 0.49350455, 0.51621614],
       [0.49638475, 0.47040293, 0.49350455, 0.51621614, 0.55494885],
       [0.47040293, 0.49350455, 0.51621614, 0.55494885, 0.5685998 ],
       [0.49350455, 0.51621614, 0.55494885, 0.5685998 , 0.56652966],
       [0.51621614, 0.55494885, 0.5685998 , 0.56652966, 0.57517026],
       [0.55494885, 0.5685998 , 0.56652966, 0.57517026, 0.57883052],
       [0.5685998 , 0.56652966, 0.57517026, 0.57883052, 0.66925685],
       [0.56652966, 0.57517026, 0.57883052, 0.66925685, 0.69889892],
       [0.57517026, 0.57883052, 0.66925685, 0.69889892, 0.70156911],
       [0.57883052, 0.66925685, 0.69889892, 0.70156911, 0.69298851],
       [0.66925685, 0.69889892, 0.70156911, 0.69298851, 0.69310852],
       [0.69889892, 0.70156911, 0.

array([[0.47040293],
       [0.49350455],
       [0.51621614],
       [0.55494885],
       [0.5685998 ],
       [0.56652966],
       [0.57517026],
       [0.57883052],
       [0.66925685],
       [0.69889892],
       [0.70156911],
       [0.69298851],
       [0.69310852],
       [0.70756953],
       [0.70924965],
       [0.74084186],
       [0.74048183],
       [0.75188263],
       [0.75245267],
       [0.75587291],
       [0.76967388],
       [0.76757373],
       [0.75077255],
       [0.75764304],
       [0.74360205],
       [0.77513426],
       [0.76514356],
       [0.76910384],
       [0.77414419],
       [0.77897453],
       [0.78197474],
       [0.78467493],
       [0.79610573],
       [0.80420629],
       [0.81131679],
       [0.80582641],
       [0.81092676],
       [0.82967808],
       [0.81794726],
       [0.83048813],
       [0.8271879 ],
       [0.83378837],
       [0.82526777],
       [0.85106957],
       [0.86340044],
       [0.86988089],
       [0.88542198],
       [0.880

Test shape (125, 5)


# FFNN

In [14]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [15]:
# Si tu y viene plano y solo tienes horizon=1, asegúrate de que sea 2D
# y = y.reshape(-1, 1)  # <- solo si y es 1D

In [16]:
# ==== 1) Modelo FFNN con Sequential.add ====

lags = X.shape[1]
horizon = y.shape[1] if y.ndim == 2 else 1  

model = keras.Sequential()
model.add(layers.Input(shape=(lags,)))          # capa de entrada
model.add(layers.Dense(128, activation='relu')) # ocultas
model.add(layers.Dense(64,  activation='relu'))
model.add(layers.Dense(horizon, activation='linear'))  # salida: horizon

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss='mse',
    metrics=['mae']
)

In [ ]:
# ==== 2) Entrenamiento ====
es = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=15, restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=32,
    verbose=1,
    callbacks=[es],
)

Epoch 1/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0453 - mae: 0.1429 - val_loss: 0.0056 - val_mae: 0.0691
Epoch 2/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9.5892e-04 - mae: 0.0239 - val_loss: 6.7953e-04 - val_mae: 0.0208
Epoch 3/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.3664e-04 - mae: 0.0144 - val_loss: 5.7575e-04 - val_mae: 0.0186
Epoch 4/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.9486e-04 - mae: 0.0137 - val_loss: 5.5789e-04 - val_mae: 0.0182
Epoch 5/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.8572e-04 - mae: 0.0131 - val_loss: 5.0285e-04 - val_mae: 0.0160
Epoch 6/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.5424e-04 - mae: 0.0125 - val_loss: 5.1082e-04 - val_mae: 0.0159
Epoch 7/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2.3892e-04 - mae: 0.0121 - val_loss: 4.9384e-04 - val_mae: 0.0160
Epoch 8/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.6961e-04 - mae: 0.0131 - val_loss: 4.7405e-04 - val_mae: 0.0151
Epoch 9

In [18]:
# ==== 3) Evaluación y uso ====
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Test MSE: {test_loss:.6f} | Test MAE: {test_mae:.6f}")

# Predicción sobre el set de test (misma forma que y)
y_pred_test = model.predict(X_test)

# Si quieres predecir el siguiente paso(s) inmediato(s) dado el último bloque de lags:
# (esto NO “camina” múltiples pasos autoregresivos; solo un forward directo de tamaño 'horizon')
x_last = X[-1].reshape(1, -1)
next_pred = model.predict(x_last)
print("Predicción próxima (shape horizon):", next_pred.ravel())


Test MSE: 0.000329 | Test MAE: 0.012592
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predicción próxima (shape horizon): [0.9305808]
